In [1]:
from GraphTsetlinMachine.graphs import Graphs
import numpy as np


In [2]:

hypervector_bits = 2
hypervector_size = 128


In [3]:
def getConnectionsOld_bad(size, index):
    x = index % size
    y = index // size

    # Define out-of-bounds placeholders
    out_of_bounds_right = "out_of_bounds_right"
    out_of_bounds_left = "out_of_bounds_left"
    out_of_bounds_down = "out_of_bounds_down"
    out_of_bounds_up = "out_of_bounds_up"

    connections = []
    directions = []

    # Up
    if y > 0:
        connections.append(x + (y - 1) * size)  # Directly above
        directions.append("up")

        # Diagonal up-left for even rows, up-right for odd rows
        if y % 2 == 0:  # Even row
            if x > 0:  # Only add if x > 0 (to avoid out-of-bound left cases)
                connections.append((x - 1) + (y - 1) * size)  # Diagonal up-left
                directions.append("up_left")
        else:  # Odd row
            if x < size - 1:  # Only add if x < size-1 (to avoid out-of-bound right cases)
                connections.append((x + 1) + (y - 1) * size)  # Diagonal up-right
                directions.append("up_right")
    else:
        connections.append(out_of_bounds_up)
        directions.append("up")

    # Left
    if x > 0:
        connections.append((x - 1) + y * size)
        directions.append("left")
    else:
        connections.append(out_of_bounds_left)
        directions.append("left")

    # Right
    if x < size - 1:
        connections.append((x + 1) + y * size)
        directions.append("right")
    else:
        connections.append(out_of_bounds_right)
        directions.append("right")

    # Down
    if y < size - 1:
        connections.append(x + (y + 1) * size)  # Directly below
        directions.append("down")

        # Diagonal down-left for even rows, down-right for odd rows
        if y % 2 == 0:  # Even row
            if x > 0:  # Only add if x > 0 (to avoid out-of-bound left cases)
                connections.append((x - 1) + (y + 1) * size)  # Diagonal down-left
                directions.append("down_left")
        else:  # Odd row
            # Correct logic for down-right: Check grid boundary conditions
            if x < size - 1 and (y + 1) * size + (x + 1) < size * size:  
                # Only add if there is a valid neighbor (within bounds)
                connections.append((x + 1) + (y + 1) * size)  # Diagonal down-right
                directions.append("down_right")
    else:
        connections.append(out_of_bounds_down)
        directions.append("down")

    return connections, directions


In [4]:
from numba import jit
import pickle

@jit
def getConnections(size, x):
        connections = []
        directions = []

        max_index = size**2-1
        right = max_index + 1
        left = max_index + 2
        down = max_index + 3
        up = max_index + 4

        if x >= size:#If x is not on top row
            connections.append(x-size)#Append top left
            directions.append("up_left")
            if x%size != size-1:#If x is not on right column
                connections.append(x-size+1) #Append top right
                directions.append("up_right")
        else:
            connections.append(up)
            directions.append("Up")
        
        if x%size != 0:#If x is not on left column
            connections.append(x-1) #Append left
            directions.append("left")
        else:
            connections.append(left)
            directions.append("Left")

        if x%size != size-1:#If x is not on right column
            connections.append(x+1)#Append right
            directions.append("right")
        else:
            connections.append(right)
            directions.append("Right")
        
        if x < (size**2)-size: #If x is not on bottom row
            connections.append(x+size)#Append bottom right
            directions.append("down_right")
            if x%size != 0:#If x is not on left column
                connections.append(x+size-1)#Append bottom left
                directions.append("down_left")
        else:
            directions.append("Down")
            connections.append(down)

        
            
        return connections, directions
            
        #if y > 0:
        #    connections.append([x, y-1])
        #    if x < self.size-1:
        #        connections.append([x+1, y-1])
        #if x > 0:
        #    connections.append([x-1, y])
        #if x < self.size-1:
        #    connections.append([x+1, y])
        #if y < self.size-1:
        #    connections.append([x, y+1])
        #    if x > 0:
        #        connections.append([x-1, y+1])

def getConnectionsExhaustive(size):
    fileConnections = f"data/connections_{size}.pkl"
    fileDirections = f"data/directions_{size}.pkl"
    try:
        with open(fileConnections, 'rb') as fCon:
            allConnections = pickle.load(fCon)
        fCon.close()
        with open(fileDirections, "rb") as fDir:
            allDirections = pickle.load(fDir)
        fDir.close()
    except:
        allConnections = []
        allDirections = []
        for i in range(size**2):
            conn, dir = getConnections(size,i)
            allConnections.append(conn)
            allDirections.append(dir)
        with open(fileConnections, "wb") as fCon:
            pickle.dump(allConnections,fCon)
        fCon.close()
        with open(fileDirections, "wb") as fDir:
            pickle.dump(allDirections, fDir)
        fDir.close()
    return allConnections, allDirections

@jit
def getConnectionsColor(size,x,board):
        connections = []
        directions = []

        max_index = size**2-1
        right = max_index + 1
        left = max_index + 2
        down = max_index + 3
        up = max_index + 4

        if x >= size:#If x is not on top row
            connections.append(x-size)#Append top left
            if board[x-size] == 1:
                directions.append("up_left_red")
            if board[x-size] == -1:
                directions.append("up_left_blue")
            else:
                directions.append("up_left_empty")
            if x%size != size-1:#If x is not on right column
                connections.append(x-size+1) #Append top right
                if board[x-size+1] == 1:
                    directions.append("up_right_red")
                elif board[x-size+1] == -1:
                    directions.append("up_right_blue")
                else:
                    directions.append("up_right_empty")
        else:
            connections.append(up)
            directions.append("Up")
        
        if x%size != 0:#If x is not on left column
            connections.append(x-1) #Append left
            if board[x-1] == 1:
                directions.append("left_red")
            elif board[x-1] == -1:
                directions.append("left_blue")
            else:
                directions.append("left_empty")
        else:
            connections.append(left)
            directions.append("Left")

        if x%size != size-1:#If x is not on right column
            connections.append(x+1)#Append right
            if board[x+1] == 1:
                directions.append("right_red")
            elif board[x+1] == -1:
                directions.append("right_blue")
            else:
                directions.append("right_empty")
        else:
            connections.append(right)
            directions.append("Right")
        
        if x < (size**2)-size: #If x is not on bottom row
            connections.append(x+size)#Append bottom right
            if board[x+size] == 1:
                directions.append("down_right_red")
            elif board[x+size] == -1:
                directions.append("down_right_blue")
            else:
                directions.append("down_right_empty")
            if x%size != 0:#If x is not on left column
                connections.append(x+size-1)#Append bottom left
                if board[x+size-1] == 1:
                    directions.append("down_left_red")
                elif board[x+size-1] == -1:
                    directions.append("down_left_blue")
                else:
                    directions.append("down_left_empty")
        else:
            directions.append("Down")
            connections.append(down)

        return connections, directions

In [5]:
# read the data from data/hex_games_1_000_000_size_7.csv
data = np.genfromtxt('data/7x7_0.csv', delimiter=',', dtype=np.int32, skip_header=1, max_rows=400000)
data


array([[-1, -1,  0, ..., -1,  1,  0],
       [ 0,  0,  0, ...,  1,  1,  1],
       [ 0,  1, -1, ...,  0,  1,  0],
       ...,
       [ 0,  1, -1, ...,  0,  1,  0],
       [ 0,  1,  1, ...,  0,  0,  1],
       [-1,  0,  1, ...,  0,  0,  0]])

In [6]:
#sumsamples = 30
#np.random.shuffle(data)
#data = data[:sumsamples]
data = np.unique(data,axis=0)
print(len(data))
# duplicate the data 100 times
#data = np.repeat(data, 1000, axis=0)

200000


In [7]:
data.shape


(200000, 51)

In [8]:
# select the first 100000 samples
data = data[:150000]

#data = np.repeat(data, 1000, axis=0)
# shuffle the data



In [9]:
# separate the last column from the rest of the data
X_data = data[:, :-1]
Y_data = data[:, -1]
X_data[0].shape

(50,)

In [10]:
# for y data we need to convert the -1 t0 0
#Y_data = np.where(Y_data == -1, 0, Y_data)



In [11]:
Y_data.mean()

0.44931333333333334

In [12]:
from sklearn.model_selection import train_test_split
# train test splitt

X_train, X_test, Y_train, Y_test = train_test_split(X_data, Y_data, test_size=0.02)
board_size = int(np.sqrt(X_data.shape[1]))
number_of_nodes = board_size*board_size
symbol_names = ['RED', 'BLUE','UP', 'DOWN', 'RIGHT','LEFT','EMPTY']

for i in range(board_size):
    symbol_names.append(f'ROW_{i}')
    symbol_names.append(f'COL_{i}')

#symbol_names.append(f"BLUE_NEIGHBOURS") 
#symbol_names.append(f"RED_NEIGHBOURS") 
symbol_names.append("CONNECTED_BLUE")
symbol_names.append("CONNECTED_RED")

max_index = number_of_nodes-1
right_index = max_index + 1
left_index = max_index + 2
down_index = max_index + 3
up_index = max_index + 4
    


In [13]:
graphs_train = Graphs(X_train.shape[0],symbols=symbol_names, hypervector_size=hypervector_size, hypervector_bits=hypervector_bits, double_hashing = False)

In [14]:
for graph_id in range(X_train.shape[0]):
    graphs_train.set_number_of_graph_nodes(graph_id, number_of_nodes+4)




graphs_train.prepare_node_configuration()
# add up, down, left, right nodes


# Add nodes to each graph
for graph_id in range(X_train.shape[0]):
    for node_id in range(number_of_nodes):
        nr_neighbours = len(getConnections(board_size, node_id)[0])
        graphs_train.add_graph_node(graph_id, node_id, nr_neighbours) 
    graphs_train.add_graph_node(graph_id, right_index, board_size)
    graphs_train.add_graph_node(graph_id, left_index, board_size)
    graphs_train.add_graph_node(graph_id, down_index,board_size)
    graphs_train.add_graph_node(graph_id, up_index, board_size)

graphs_train.prepare_edge_configuration()



In [15]:
getConnections(3, 2)

([12, 1, 9, 5, 4], ['Up', 'left', 'Right', 'down_right', 'down_left'])

In [16]:
a= range(1,2)
for i in a:
    print(i)

1


In [17]:

allConnections, allDirections = getConnectionsExhaustive(board_size)
for graph_id in range(X_train.shape[0]):
    for node_id in range(number_of_nodes+4):
        edge_type = 0
        if node_id<number_of_nodes:
            
            neighbors, directions = getConnectionsColor(board_size,node_id,X_train[graph_id])
            for neighbor_id,dir in zip(neighbors,directions):
                graphs_train.add_graph_node_edge(graph_id, node_id, neighbor_id,dir)

          

        

            node_value = X_train[graph_id, node_id]

            redCount = 0
            blueCount = 0

            for i in neighbors:
                if i<board_size**2:
                    if X_train[graph_id,i] == 1:
                        redCount += 1
                    elif X_train[graph_id,i] == -1:
                        blueCount += 1
            
            #if node_value != 0:
            #    if redCount > 1:
            #        graphs_train.add_graph_node_property(graph_id,node_id, f"RED_NEIGHBOURS")
            #    if blueCount > 1:
            #        graphs_train.add_graph_node_property(graph_id,node_id, f"BLUE_NEIGHBOURS")

        
            if node_value == 1:
                graphs_train.add_graph_node_property(graph_id, node_id, 'RED')
                if (redCount > 1) or (redCount != 0 and down_index in neighbors) or (redCount != 0 and up_index in neighbors):
                    graphs_train.add_graph_node_property(graph_id, node_id, "CONNECTED_RED")
                    
            elif node_value == -1:
                graphs_train.add_graph_node_property(graph_id, node_id, 'BLUE')
                if (blueCount > 1) or (blueCount != 0 and left_index in neighbors) or (blueCount != 0 and right_index in neighbors):
                    graphs_train.add_graph_node_property(graph_id, node_id, "CONNECTED_BLUE")

            elif node_value == 0:
                pass
                #graphs_train.add_graph_node_property(graph_id, node_id, 'EMPTY')
                #if left_index in neighbors:
                #    graphs_train.add_graph_node_property(graph_id, node_id, 'LEFT_EMPTY')
                #elif right_index in neighbors:
                #    graphs_train.add_graph_node_property(graph_id, node_id, 'RIGHT_EMPTY')
                #if up_index in neighbors:
                #    graphs_train.add_graph_node_property(graph_id, node_id, 'UP_EMPTY')
                #elif down_index in neighbors:
                #    graphs_train.add_graph_node_property(graph_id, node_id, 'DOWN_EMPTY')
                    

                
            row = node_id // board_size
            col = node_id % board_size
            
                
    
        #    graphs_train.add_graph_node_property(graph_id, node_id, f'ROW_{row}COL_{col}')
            for r in range(row+1):
                graphs_train.add_graph_node_property(graph_id, node_id, f'ROW_{r}')
            for c in range(col+1):
                graphs_train.add_graph_node_property(graph_id, node_id, f'COL_{c}')
        if node_id == right_index:
            neighbors = [i for i in range(board_size-1,board_size*board_size,board_size)]
            edge_type = 'Right'
            for neighbor_id in neighbors:
                graphs_train.add_graph_node_edge(graph_id, node_id, neighbor_id,edge_type)
            graphs_train.add_graph_node_property(graph_id, node_id, 'RIGHT')
        if node_id == left_index:
            neighbors = [i for i in range(0,board_size*board_size,board_size)]
            edge_type = 'Left'
            for neighbor_id in neighbors:
                graphs_train.add_graph_node_edge(graph_id, node_id, neighbor_id,edge_type)
            graphs_train.add_graph_node_property(graph_id, node_id, 'LEFT')
        if node_id == down_index:
            neighbors = [i for i in range(board_size*board_size-board_size,board_size*board_size,1)]
            edge_type = 'Down'
            for neighbor_id in neighbors:
                graphs_train.add_graph_node_edge(graph_id, node_id, neighbor_id,edge_type)
            graphs_train.add_graph_node_property(graph_id, node_id, 'DOWN')
        if node_id == up_index:
            neighbors = [i for i in range(board_size)]
            edge_type = 'Up'
            for neighbor_id in neighbors:
                graphs_train.add_graph_node_edge(graph_id, node_id, neighbor_id,edge_type)
            graphs_train.add_graph_node_property(graph_id, node_id, 'UP')



In [18]:
[i for i in range(board_size-1,board_size*board_size,board_size)]

[6, 13, 20, 27, 34, 41, 48]

In [19]:
board_size

7

In [20]:
graphs_train.encode()

In [21]:
X_test.shape[0]

3000

In [22]:

graphs_test = Graphs(X_test.shape[0], init_with=graphs_train)


In [23]:

for graph_id in range(X_test.shape[0]):
    graphs_test.set_number_of_graph_nodes(graph_id, number_of_nodes+4)


 
graphs_test.prepare_node_configuration()


for graph_id in range(X_test.shape[0]):
 
    for node_id in range(number_of_nodes):
        nr_neighbours = len(getConnections(board_size, node_id)[0])
        graphs_test.add_graph_node(graph_id, node_id, nr_neighbours)  
    graphs_test.add_graph_node(graph_id, right_index, board_size)
    graphs_test.add_graph_node(graph_id, left_index, board_size)
    graphs_test.add_graph_node(graph_id, down_index,board_size)
    graphs_test.add_graph_node(graph_id, up_index, board_size)

graphs_test.prepare_edge_configuration()

In [24]:


for graph_id in range(X_test.shape[0]):
    for node_id in range(number_of_nodes+4):
     
        neighbors, directions = getConnectionsColor(board_size,node_id,X_test[graph_id]) 
        edge_type = 0
        if node_id<number_of_nodes:
            for neighbor_id, dir in zip(neighbors,directions):
                graphs_test.add_graph_node_edge(graph_id, node_id, neighbor_id,dir)
     
    
            node_value = X_test[graph_id, node_id]
            
            redCount = 0
            blueCount = 0

            for i in neighbors:
                if i<board_size**2:
                    if X_test[graph_id,i] == 1:
                        redCount += 1
                    elif X_test[graph_id,i] == -1:
                        blueCount += 1

            #if node_value != 0:
            #    if redCount > 1:
            #        graphs_train.add_graph_node_property(graph_id,node_id, f"RED_NEIGHBOURS")
            #    if blueCount > 1:
            #        graphs_train.add_graph_node_property(graph_id,node_id, f"BLUE_NEIGHBOURS")


    
            if node_value == 1:
                graphs_test.add_graph_node_property(graph_id, node_id, 'RED')
                if (redCount > 1) or (redCount != 0 and down_index in neighbors) or (redCount != 0 and up_index in neighbors):
                    graphs_test.add_graph_node_property(graph_id, node_id, "CONNECTED_RED")
        
            elif node_value == -1:
                graphs_test.add_graph_node_property(graph_id, node_id, 'BLUE')
                if (blueCount > 1) or (blueCount != 0 and left_index in neighbors) or (blueCount != 0 and right_index in neighbors):
                    graphs_test.add_graph_node_property(graph_id, node_id, "CONNECTED_BLUE")

            elif node_value == 0:
                pass
                #graphs_test.add_graph_node_property(graph_id, node_id, 'EMPTY')

       
                #if left_index in neighbors:
                #    graphs_test.add_graph_node_property(graph_id, node_id, 'LEFT_EMPTY')
                #elif right_index in neighbors:
                #    graphs_test.add_graph_node_property(graph_id, node_id, 'RIGHT_EMPTY')
                #if up_index in neighbors:
                #    graphs_test.add_graph_node_property(graph_id, node_id, 'UP_EMPTY')
                #elif down_index in neighbors:
                #    graphs_test.add_graph_node_property(graph_id, node_id, 'DOWN_EMPTY')

            row = node_id // board_size
            col = node_id % board_size

                 
          #  graphs_test.add_graph_node_property(graph_id, node_id, f'ROW_{row}COL_{col}')
            for c in range(col+1):
                graphs_test.add_graph_node_property(graph_id, node_id, f'COL_{c}')
            for r in range(row+1):
                graphs_test.add_graph_node_property(graph_id, node_id, f'ROW_{r}')

       
        
    
        if node_id == right_index:
            neighbors = [i for i in range(board_size-1,board_size*board_size,board_size)]
            edge_type = 'Right'
            for neighbor_id in neighbors:
                graphs_test.add_graph_node_edge(graph_id, node_id, neighbor_id,edge_type)
            graphs_test.add_graph_node_property(graph_id, node_id, 'RIGHT')
        if node_id == left_index:
            neighbors = [i for i in range(0,board_size*board_size,board_size)]
            edge_type = 'Left'
            for neighbor_id in neighbors:
                graphs_test.add_graph_node_edge(graph_id, node_id, neighbor_id,edge_type)
            graphs_test.add_graph_node_property(graph_id, node_id, 'LEFT')
        if node_id == down_index:
            neighbors = [i for i in range(board_size*board_size-board_size,board_size*board_size,1)]
            edge_type = 'Down'
            for neighbor_id in neighbors:
                graphs_test.add_graph_node_edge(graph_id, node_id, neighbor_id,edge_type)
            graphs_test.add_graph_node_property(graph_id, node_id, 'DOWN')
        if node_id == up_index:
            neighbors = [i for i in range(board_size)]
            edge_type = 'Up'
            for neighbor_id in neighbors:
                graphs_test.add_graph_node_edge(graph_id, node_id, neighbor_id,edge_type)
            graphs_test.add_graph_node_property(graph_id, node_id, 'UP')
       

In [25]:
graphs_test.encode()

In [26]:
# pickle dump all the  (graphs_train, graphs_test, X_train, Y_train, X_test, Y_test)
import pickle
with open('data.pkl', 'wb') as f:
    pickle.dump((graphs_train, graphs_test, X_train, Y_train, X_test, Y_test), f)


In [27]:
from GraphTsetlinMachine.tm import MultiClassGraphTsetlinMachine

In [28]:
number_of_nodes = board_size*board_size
number_of_clauses = 50
T = number_of_clauses*1.5971422963103452
depth = 2
s = 0.051723993754933015
message_size = 64
message_bits = 2
max_included_literals = 20
number_of_states = 200
epochs = 40

In [29]:
# Verify sizes and shapes
print(f"Number of Graphs: {graphs_train.number_of_graphs}")
print(f"Number of Nodes: {graphs_train.number_of_nodes}")
print(f"X Shape: {graphs_train.X.shape}")
print(f"Edge Array Shape: {graphs_train.edge.shape}")

# Check indices
print(f"Node Index Array: {graphs_train.node_index}")
print(f"Edge Index Array: {graphs_train.edge_index}")


Number of Graphs: 147000
Number of Nodes: 7791000
X Shape: (7791000, 8)
Edge Array Shape: (43512000, 2)
Node Index Array: [      0      53     106 ... 7790841 7790894 7790947]
Edge Index Array: [       0        4        9 ... 43511979 43511986 43511993]


In [30]:
tm = MultiClassGraphTsetlinMachine(
    number_of_clauses, T, s, depth=depth, message_size = message_size,
    message_bits = message_bits, number_of_state_bits = 8,max_included_literals = max_included_literals,
    	grid=(16*13,1,1),
  	block=(128,1,1)
    

)


Initialization of sparse structure.


In [31]:
Y_test.mean()

0.42933333333333334

In [32]:


#torch.cuda.empty_cache()<
#torch.cuda.reset_max_memory_allocated() # Clear GPU memory
for i in range(epochs):
    tm.fit(graphs_train, Y_train, epochs=1, incremental=True)
  
    result_test = 100 * ( tm.predict(graphs_test) == Y_test).mean()
    result_train = 100 * (tm.predict(graphs_train) == Y_train).mean()
    
    print("#%d Testing Accuracy: %.2f%% Training Accuracy: %.2f%%" % (i+1, result_test, result_train))

    if result_test == 100:
        break
    
   
  

c:\Users\jonin\.conda\envs\hex\Lib\site-packages\GraphTsetlinMachine\tm.py:286: UserWarning: The CUDA compiler succeeded, but said the following:
kernel.cu

  mod_prepare = SourceModule(parameters + kernels.code_header + kernels.code_prepare, no_extern_c=True)
c:\Users\jonin\.conda\envs\hex\Lib\site-packages\GraphTsetlinMachine\tm.py:293: UserWarning: The CUDA compiler succeeded, but said the following:
kernel.cu

  mod_update = SourceModule(parameters + kernels.code_header + kernels.code_update, no_extern_c=True)
c:\Users\jonin\.conda\envs\hex\Lib\site-packages\GraphTsetlinMachine\tm.py:300: UserWarning: The CUDA compiler succeeded, but said the following:
kernel.cu

  mod_evaluate = SourceModule(parameters + kernels.code_header + kernels.code_evaluate, no_extern_c=True)
c:\Users\jonin\.conda\envs\hex\Lib\site-packages\GraphTsetlinMachine\tm.py:325: UserWarning: The CUDA compiler succeeded, but said the following:
kernel.cu

  mod_transform = SourceModule(parameters + kernels.code_hea

#1 Testing Accuracy: 94.30% Training Accuracy: 94.39%
#2 Testing Accuracy: 92.40% Training Accuracy: 92.58%
#3 Testing Accuracy: 93.63% Training Accuracy: 93.58%
#4 Testing Accuracy: 94.53% Training Accuracy: 95.36%
#5 Testing Accuracy: 93.60% Training Accuracy: 94.73%


KeyboardInterrupt: 

In [33]:
tm.score(graphs_test)

array([[-219,  251],
       [-238,  262],
       [ 211, -249],
       ...,
       [  15,  -23],
       [-248,  242],
       [ 317, -335]])

In [34]:
result_test

93.60000000000001

In [35]:
result_train

94.72517006802721

In [36]:
predictions = tm.predict(graphs_test)

In [37]:
predictions.mean()

0.44533333333333336

In [38]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(Y_test,predictions))


              precision    recall  f1-score   support

           0       0.97      0.94      0.95      1712
           1       0.92      0.95      0.94      1288

    accuracy                           0.95      3000
   macro avg       0.94      0.95      0.94      3000
weighted avg       0.95      0.95      0.95      3000



In [39]:
confusion_matrix(Y_test,predictions)

array([[1606,  106],
       [  58, 1230]], dtype=int64)

In [40]:
graphs_train.hypervectors

array([[ 56,  64],
       [ 37,  20],
       [122,  71],
       [ 62, 101],
       [ 49,  78],
       [126,  18],
       [ 71, 100],
       [116, 109],
       [ 87,  78],
       [100, 126],
       [ 23,  59],
       [ 60, 100],
       [ 88,  38],
       [102,  13],
       [ 77,  66],
       [ 52,  42],
       [ 24,  80],
       [ 35,  86],
       [109,  13],
       [ 20, 115],
       [123,  20],
       [  1,  79],
       [ 33, 104]], dtype=uint32)

In [41]:
weights = tm.get_state()[1].reshape(2, -1)
for i in range(tm.number_of_clauses):
        print("Clause #%d W:(%d %d)" % (i, weights[0,i], weights[1,i]), end=' ')
        l = []
        for k in range(hypervector_size * 2):
            if tm.ta_action(0, i, k):
                if k < hypervector_size:
                    l.append("x%d" % (k))
                else:
                    l.append("NOT x%d" % (k - hypervector_size))
        print(" AND ".join(l))


Clause #0 W:(1 -5) NOT x1 AND NOT x18 AND NOT x62 AND NOT x66 AND NOT x77 AND NOT x79 AND NOT x101
Clause #1 W:(8 -1) x60 AND x78 AND x87 AND x100 AND x109 AND x116 AND x126 AND NOT x18 AND NOT x62 AND NOT x101 AND NOT x115
Clause #2 W:(46 -57) x33 AND x104 AND NOT x1 AND NOT x35 AND NOT x42 AND NOT x52 AND NOT x79 AND NOT x86 AND NOT x115
Clause #3 W:(21 6) x23 AND x59 AND NOT x42 AND NOT x49 AND NOT x52 AND NOT x102
Clause #4 W:(25 -17) NOT x1 AND NOT x24 AND NOT x56 AND NOT x64 AND NOT x66 AND NOT x77 AND NOT x79 AND NOT x80 AND NOT x123
Clause #5 W:(10 1) x1 AND x13 AND x20 AND x60 AND x79 AND x102 AND NOT x33 AND NOT x104
Clause #6 W:(40 -34) NOT x37 AND NOT x60 AND NOT x100 AND NOT x102 AND NOT x126
Clause #7 W:(43 -45) x33 AND x56 AND x64 AND x100 AND x104 AND x126 AND NOT x13 AND NOT x20 AND NOT x35 AND NOT x42 AND NOT x52 AND NOT x86 AND NOT x102 AND NOT x115
Clause #8 W:(22 -21) NOT x33 AND NOT x37 AND NOT x56 AND NOT x64 AND NOT x104
Clause #9 W:(-47 40) x20 AND x24 AND x66 

In [42]:
import matplotlib.pyplot as plt
from matplotlib import colors
from tqdm import tqdm

In [43]:
def scale(X, x_min, x_max):
	nom = (X - X.min()) * (x_max - x_min)
	denom = X.max() - X.min()
	denom = denom + (denom == 0)
	return x_min + nom / denom


def scale_image(img):
	if len(img.shape) == 3:
		for ch in range(3):
			img[..., ch] = scale(img[..., ch], 0, 1)
	else:
		img = scale(img, 0, 1)

	return img


weights = tm.get_state()[1].reshape(tm.number_of_outputs, tm.number_of_clauses)

# Get Literals insymbol format
clause_literals = tm.get_clause_literals(graphs_train.hypervectors)
num_symbols = len(graphs_train.symbol_id)

# get output of each clause at each node
clause_outputs, class_sums = tm.transform_nodewise(graphs_test)


In [44]:
clause_outputs

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 1, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 

In [45]:

weights = tm.get_state()[1].reshape(2, -1)

print(f"Symbol hypervectors: \n{graphs_train.hypervectors}\n\n")
print(f"Clause hypervectors: \n{tm.hypervectors=}\n\n")

print("Clause in Hyperliterals format:")
for clause in range(tm.number_of_clauses):
	print(f"Clause {clause} [{weights[0, clause]:>4d} {weights[1, clause]:>4d}]", end=": ")
	print(*[int(tm.ta_action(depth=0, clause=clause, ta=i)) for i in range(graphs_train.hypervector_size * 2)])
print()

print("Messages as hypervectors:")
for clause in range(tm.number_of_clauses):
	print(f"Clause {clause} [{weights[0, clause]:>4d} {weights[1, clause]:>4d}]", end=": ")
	print(*[int(tm.ta_action(depth=1, clause=clause, ta=i)) for i in range(tm.message_size * 2)])

# Get Clauses in symbols format and Messages in clause_indices format
clause_literals = tm.get_clause_literals(graphs_train.hypervectors)
message_clauses = tm.get_messages(1, len(graphs_train.edge_type_id))
num_symbols = len(graphs_train.symbol_id)

# Create symbol_id to symbol_name dictionary for printing symbol names
symbol_dict = dict((v, k) for k, v in graphs_train.symbol_id.items())

print("Actual clauses:")
for clause in range(tm.number_of_clauses):
	print(f"Clause {clause} [{weights[0, clause]:>4d} {weights[1, clause]:>4d}]", end=": ")
	for literal in range(num_symbols):
		if clause_literals[clause, literal] > 0:
			print(f"{clause_literals[clause, literal]}{symbol_dict[literal]}", end=" ")

		if clause_literals[clause, literal + num_symbols] > 0:
			print(f"~{clause_literals[clause, literal + num_symbols]}{symbol_dict[literal]}", end=" ")

	print("")

# Print messages for each edge type
for edge_type in range(len(graphs_train.edge_type_id)):
	print(f"Actual Messages for {edge_type=}:")

	for msg in range(tm.number_of_clauses):
		print(f"Message {msg} ", end=": ")

		for clause in range(tm.number_of_clauses):
			if message_clauses[edge_type, msg, clause] == 1:
				print(f"C:{clause}(", end=" ")

				for literal in range(num_symbols):
					if clause_literals[clause, literal] > 0:
						print(f"{clause_literals[clause, literal]}{symbol_dict[literal]}", end=" ")

					if clause_literals[clause, literal + num_symbols] > 0:
						print(f"~{clause_literals[clause, literal + num_symbols]}{symbol_dict[literal]}", end=" ")

				print(")", end=" ")

			if message_clauses[edge_type, msg, tm.number_of_clauses + clause] == 1:
				print(f"~C:{clause}(", end=" ")

				for literal in range(num_symbols):
					if clause_literals[clause, literal] > 0:
						print(f"{clause_literals[clause, literal]}{symbol_dict[literal]}", end=" ")

					if clause_literals[clause, literal + num_symbols] > 0:
						print(f"~{clause_literals[clause, literal + num_symbols]}{symbol_dict[literal]}", end=" ")

				print(")", end=" ")

		print("")

Symbol hypervectors: 
[[ 56  64]
 [ 37  20]
 [122  71]
 [ 62 101]
 [ 49  78]
 [126  18]
 [ 71 100]
 [116 109]
 [ 87  78]
 [100 126]
 [ 23  59]
 [ 60 100]
 [ 88  38]
 [102  13]
 [ 77  66]
 [ 52  42]
 [ 24  80]
 [ 35  86]
 [109  13]
 [ 20 115]
 [123  20]
 [  1  79]
 [ 33 104]]


Clause hypervectors: 
tm.hypervectors=array([[44, 40],
       [46,  7],
       [26, 13],
       [37, 16],
       [26, 27],
       [56,  1],
       [59, 33],
       [25, 33],
       [21, 15],
       [28, 13],
       [47,  6],
       [53, 38],
       [ 4, 48],
       [26, 18],
       [58, 62],
       [49, 20],
       [49, 12],
       [42, 34],
       [35, 51],
       [ 3, 11],
       [55, 50],
       [46, 40],
       [34,  9],
       [16,  1],
       [62, 35],
       [23, 32],
       [13, 46],
       [15,  3],
       [47, 60],
       [22, 28],
       [61, 20],
       [51,  0],
       [59, 33],
       [54,  9],
       [13, 45],
       [55, 48],
       [26, 17],
       [16, 30],
       [48, 62],
       [59, 55],
    